In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import os
from iris_plant_visualizer_C_space import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid, IrisInConfigurationSpace
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver

In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_file = "assets/oneDOF_iiwa7_with_box_collision.sdf"
with open(oneDOF_iiwa_file, 'r') as f:
    oneDOF_iiwa_string = f.read()
box_asset_file = "assets/box_small.urdf"
with open(box_asset_file, 'r') as f:
    box_asset_string = f.read()

models = []
models.append(parser.AddModelFromFile(box_asset_file))
models.append(parser.AddModelFromFile(oneDOF_iiwa_file, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_file,  'left_sweeper'))

oneDOF_iiwa_file = "assets/b"

locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)

plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1

do_viz = True
# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


In [5]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [7]:
# Some seedpoints
seed_points_q = np.array([   
                              [-0.6, 0.55],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.5,-1.08],
                              [-1.06, 0.48]
                              ])
visualizer.plot_cspace_points(seed_points_q, "/iris_seed_points", radius = 0.02)

default_alpha = 0.2
colors_dict = {
    0: Rgba(0.565, 0.565, 0.565, default_alpha), # gray
    1: Rgba(0.118, 0.533, 0.898, default_alpha), # bluish
    2: Rgba(1,     0.757, 0.027, default_alpha), # gold
    3: Rgba(0,     0.549, 0.024, default_alpha), # green   
    4: Rgba(0.055, 0.914, 0.929, default_alpha), # teal 
}


In [10]:
iris_regions = []
iris_ellipses = []
collisions = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = False
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

context_for_iris = visualizer.task_space_diagram_context
for i, q in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context_for_iris), q)
    r = IrisInConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context_for_iris),
                                         iris_options)
    iris_regions.append((r, colors_dict[i]))
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05


In [11]:
visualizer.add_group_of_regions_to_visualization(iris_regions, "/uncertified-iris", 
                                                            wireframe = False, opacity = 0.2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-05
Warning for adding constraints: zero or small (< 1e